In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import credentials
import requests
import datetime as dt
#pulling data API
key =credentials.login['key_isaham']  # extract password

In [2]:
def sector_filter(sector):

  # pulling data from api
  URL = 'https://admin.isaham.my/api/stocks/list'
  r = requests.get(url = URL) 
  data = r.json() 
  df = pd.DataFrame(data['stocks'],columns = ['code','symbol','name','primary-sector','secondary-sector'])

  wl = df[df["primary-sector"]==sector]
  wl = list(wl.symbol)

  return(wl)

In [4]:
def strategy_plan(wl,start_year,end_year):
    wp_acc=[]
    wp_only=[]
    total_trade=[]
    wr = []
    Name_stock = []
    for m,stock in enumerate (wl):
        #print("----------------PROCESSING {}------------------".format(stock))
        #symbol = "UNISEM"  # which stock to pull
        symbol = stock    

        response = requests.get("https://admin.isaham.my/api/chart?stock={}&key={}".format(symbol,key))
        data = response.json()
        #print(data)
        plt.style.use('fivethirtyeight')

        %config InlineBackend.figure_format = 'retina'
        %matplotlib inline

        # makes plots bigger
        plt.rcParams["figure.figsize"] = (20,8)

        #Tuning OB and OS level
        ob_cross=85
        os_cross=35

        df= pd.DataFrame.from_dict(data)
        df= df[['c','d','h','l','o','v']]
        df['d'] = pd.to_datetime(df['d'], dayfirst= True )
        #identify MA50
        n=50
        ma50=[] 

        for i in range(len(df["c"])-(n-1)):
            ma = round(sum(df["c"][i:i+(n)])/n,4)
            #print(ma)
            ma50.append(ma)

        df["MA50"] = pd.Series(ma50)
        df["MA50"] = df["MA50"].shift(n-1)

        #filter uptrend price above ma50
        # Calculating the gap of both MA
        df["MAgap"] = df["c"] -  df["MA50"] 

        df.set_index('d',inplace=True,drop=True)
        df.index.name = None

        #create signal for uptrend price above ma50 
        #result not accurate for stochastic when just filter uptrend only
        df["Uptrend"]=df['MAgap']>0
        Uptrend=df["Uptrend"]
        Uptrend_signal=Uptrend[Uptrend==True]
        #signal stochastic
        #Create the "L14" column in the DataFrame
        df['L14'] = df['l'].rolling(window=14).min()

        #Create the "H14" column in the DataFrame
        df['H14'] = df['h'].rolling(window=14).max()

        #Create the "%K" column in the DataFrame
        df['%K'] = 100*((df['c'] - df['L14']) / (df['H14'] - df['L14']) )

        #Create the "%D" column in the DataFrame
        df['%D'] = df['%K'].rolling(window=3).mean()

        #Create a column in the DataFrame showing "TRUE" if buy entry signal is given and "FALSE" otherwise. 
        #A buy is initiated when the %K line crosses up through the %D line and the value of the oscillator is below 20 
        df['Buy OS'] = ((df['%K'] > df['%D']) & (df['%K'].shift(1) < df['%D'].shift(1))) & (df['%D'] < os_cross) & (Uptrend_signal)

        #Create a column in the DataFrame showing "TRUE" if sell entry signal is given and "FALSE" otherwise. 
        #A sell is initiated when the %K line crosses down through the %D line and the value of the oscillator is above 80 
        df['Sell OB'] = ((df['%K'] < df['%D']) & (df['%K'].shift(1) > df['%D'].shift(1))) & (df['%D'] > ob_cross) 

        overbought =df['Sell OB']
        oversold =df['Buy OS']
        date_os=list(oversold[oversold == True].index)
        date_ob=list(overbought[overbought == True].index)
        
            # transaction recording for stochastic trading plan
        buy = []
        sell = []
        sdate = []
        bdate = []
        wr = []
        pct = []
        for n,i in enumerate(date_os):
            #print(n,"Buy:", i, "Price:", df['c'].loc[i])
            buy.append(df['c'].loc[i])
            bdate.append(i)
            r=[]
            #print("before", len(r))
            for m,k in enumerate(date_ob):
                if k>i:
                    r.append(m)
                    #print('after',len(r))
                    if len(r) ==1:
                        #print(m,"Sell :", k, "Price:",df['c'].loc[k])
                        #print("Percentage change:",(((df['c'].loc[k]/df['c'].loc[i])-1)*100) )
                        sell.append(df['c'].loc[k])
                        sdate.append(k)
                        pct_change=((df['c'].loc[k]/df['c'].loc[i])-1)*100
                        pct.append(((df['c'].loc[k]/df['c'].loc[i])-1)*100)
                    elif len(r)>1:
                        pass
        diff =len(buy)-len(sell)
        # Checking for final transaction, make sure it is sell, as we want to close the position
        if len(buy) == len(sell):
            #print("Yes \n", "Buy:", len(buy), "Sell:", len(sell))
            #print("Position closed")
            pass

        elif len(buy) != len(sell):
            #print("No, \n", "Buy:{}, Sell:{}, Closing the position".format(len(buy),len(sell)))
            del(buy[-diff:])
            del(bdate[-diff:])
            #print("Position closed")
        dct = {
        "Buy_Date": bdate,
        "Buy": buy,
        "Sell": sell,
        "Sell_Date": sdate,
        "Pct_change": pct }   
        trans = pd.DataFrame(dct)


        #define profit or loss
        for i in range(len(trans)):
            if trans["Pct_change"][i] > 0:
                wr.append(1)
            else:
                wr.append(0)

        # define name of stock
        for s in range(len(trans)):
            Name_stock.append(symbol) 
        trans["wr"] = pd.Series(wr)
        trans['Stock'] = pd.Series(Name_stock)

        trans.set_index('Buy_Date', inplace=True)

        #set date for period of backtest
        #trans=trans['01-01-2020':]
        
        date1 = "01-01-{}".format(start_year)  
        date2 = "01-01-{}".format(end_year)  
        #date3 =
        trans=trans[date1:date2]
        #print(trans)

        win_only=trans[trans["wr"]==1]
        #print(win_only)
        net_pct_win=round(sum(win_only['Pct_change']),4)
        wp_only.append(net_pct_win)
        #print(net_pct_win)

        #calculate net percentage
        net_pct=round(sum(trans['Pct_change']),4)
        wp_acc.append(net_pct)

        #calculate number of trade record
        num_trade=len(trans['wr'])
        total_trade.append(num_trade)

        #print('Stock :{}'.format(symbol))
        #print('When Overbought cross>{}.'.format(ob_cross))
        #print('When Oversold cross<{}.'.format(os_cross))
        #print("Winning rate at {}% for {} trades .".format(net_pct, len(trans["wr"])))
        #print("----------------DONE {}------------------".format(symbol)) 
    #print('######################### DONE DATA PREP ###########################')
    return(wp_acc,wp_only,total_trade,date1,date2)

        



In [5]:
def average_wr(wp_acc,wp_only,total_trade,date1,date2):

    avg_2 = round(sum(wp_acc)/len(wp_acc),4)
    total_win =sum(wp_only)/len(wp_only)
    total_num_trade =round(sum(total_trade),4)
    print("Overall Winning Rate from {} until {} : {}%".format(date1,date2,avg_2))
    print('Total number of trade : {}'.format(total_num_trade))
    print("Average Winning Rate for win only : {}%".format(total_win)) 
    print("############################## Finish #################################")
    return avg_2

In [6]:
def winning_rate(wl):
    
    wp_acc,wp_only,total_trade,date1,date2=strategy_plan(wl,start_year=2020,end_year=2021)
    avg_2=average_wr(wp_acc,wp_only,total_trade,date1,date2)
    #wp_acc,wp_only,total_trade=label_column(trans,symbo)
    
    return wp_acc,wp_only,total_trade,avg_2,date1,date2

In [7]:
#sectors = ["Technology", "Healthcare", "Construction"]

sectors = ['Healthcare',
         'Telecommunications & Media',
         'Technology',
         'Utilities',
        'Energy',
        'Construction']
for sector in sectors:
    wl = sector_filter(sector)
    print(f"{sector}:")
    wp_acc,wp_only,total_trade,avg_2,date1,date2= winning_rate(wl)
    
   

Healthcare:
Overall Winning Rate for 01-01-2020 until 01-01-2021 : 100.4283%
Total number of trade : 95
Average Winning Rate for win only : 125.20988333333334%
############################## Finish #################################
Telecommunications & Media:
Overall Winning Rate for 01-01-2020 until 01-01-2021 : 114.1105%
Total number of trade : 165
Average Winning Rate for win only : 137.50732258064517%
############################## Finish #################################
Technology:
Overall Winning Rate for 01-01-2020 until 01-01-2021 : 169.2336%
Total number of trade : 572
Average Winning Rate for win only : 205.11016744186045%
############################## Finish #################################
Utilities:
Overall Winning Rate for 01-01-2020 until 01-01-2021 : 39.8938%
Total number of trade : 80
Average Winning Rate for win only : 65.85570769230769%
############################## Finish #################################
Energy:
Overall Winning Rate for 01-01-2020 until 01-01-2